In [1]:
!pip uninstall numpy -y
!pip uninstall torch torchvision torchaudio -y
!pip uninstall ultralytics -y
!pip uninstall accelerate -y
!pip uninstall ultralytics-thop -y
!pip uninstall xformers -y

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: ultralytics 8.3.141
Uninstalling ultralytics-8.3.141:
  Successfully uninstalled ultralytics-8.3.141
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
Found existing installation: ultralytics-thop 2.0.14
Uninstalling ultralytics-thop-2.0.14:
  Successfully uninstalled ultralytics-thop-2.0.14


In [2]:
!pip install numpy==1.26.4

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)


In [3]:
!pip install --upgrade diffusers

In [4]:
!pip install --upgrade transformers

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [6]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [7]:
!pip install --upgrade accelerate
!pip install --upgrade safetensors
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu118

  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.7.0-py3-none-any.whl (362 kB)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     --------------------------------------- 10.8/10.8 MB 74.5 MB/s eta 0:00:00


In [8]:
import torch
import torchvision
import torchaudio

print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)
print(torch.cuda.is_available())

2.4.0+cu118
0.19.0+cu118
2.4.0+cu118
True


In [9]:
import xformers
print(xformers.__version__)

0.0.27.post2+cu118


In [13]:
from diffusers import StableDiffusionPipeline
import torch
from pathlib import Path
from tqdm import tqdm
import random
import shutil

In [12]:
!pip install -U "triton-windows<3.4"

   ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
   - -------------------------------------- 1.0/41.9 MB 17.1 MB/s eta 0:00:03
   -- ------------------------------------- 2.4/41.9 MB 5.4 MB/s eta 0:00:08
   ---------- ----------------------------- 11.3/41.9 MB 18.1 MB/s eta 0:00:02
   ------------ --------------------------- 12.8/41.9 MB 17.1 MB/s eta 0:00:02
   ----------------------- ---------------- 24.1/41.9 MB 23.1 MB/s eta 0:00:01
   ---------------------------------------  40.9/41.9 MB 33.4 MB/s eta 0:00:01
   ---------------------------------------- 41.9/41.9 MB 28.7 MB/s eta 0:00:00


In [ ]:
h_token = "Access Token"  # replace with your Hugging Face token
pipe = StableDiffusionPipeline.from_pretrained(
    "Lykon/dreamshaper-8",
    torch_dtype=torch.float16,
    use_auth_token=h_token,
    use_safetensors=True
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [22]:
output_dir = Path("mask_images")
image_size = (640, 640)
batch_size = 4
images_per_class = 500
num_inference_steps = 25
genders = [
    "asian man", "asian woman", "white man", "white woman",
    "black man", "black woman", "latino man", "latino woman",
    "middle eastern man", "middle eastern woman"
]
poses = [
    "front view",
    "side view",
    "three-quarter view",
    "looking up",
    "looking down",
    "head tilted left",
    "head tilted right"
]
backgrounds = [
    "indoor", "outdoor", "office", "street", "cafe",
    "school", "subway", "park", "shopping mall",
    "restaurant", "library"
]
clothes = [
    "in casual clothes", "wearing a hoodie", "in a business suit",
    "in a school uniform", "wearing sportswear", "wearing traditional clothes"
]
mask_colors = ["white", "beige", "blue", "black", "gray", "pink"]
expressions = [
    "smiling",                 # 미소 짓고 있는 표정입니다.
    "neutral expression",      # 무표정(감정 없는) 모습입니다.
    "frowning",                # 찡그린(불쾌한, 언짢은) 표정입니다.
    "surprised expression",    # 놀란 듯한 표정입니다.
    "serious expression",      # 진지한(심각한) 표정입니다.
    "happy expression"         # 행복해 보이는 표정입니다.
]
shot_types = [
    "close-up face",             # 얼굴 부분만 가까이 찍은 사진입니다.
    "portrait, head and shoulders", # 얼굴과 어깨 부분까지 나타낸 초상화 스타일 사진입니다.
    "upper body portrait",       # 상체(가슴 위쪽)까지 나타낸 사진입니다.
    "full body portrait"         # 전신(머리부터 발끝까지)을 나타낸 사진입니다.
]
class_prompt_templates = {
    # 클래스 이름 'mask_on': 마스크를 착용한 얼굴 이미지를 생성하기 위한 지시문(프롬프트)입니다.
    "mask_on": (
        "a {shot} of a {gender} wearing a {mask_color} face mask covering mouth and nose properly, "
        "{expression}, {pose}, {background}, {clothes}, realistic, high-quality"
    ),

    # 클래스 이름 'no_mask': 마스크를 착용하지 않은 얼굴 이미지를 생성하기 위한 지시문입니다.
    "no_mask": (
        "a {shot} of a {gender} without mask, {expression}, {pose}, "
        "{background}, {clothes}, realistic, high-quality"
    )
}

negative_prompts = {
    "mask_on": "blurry, deformed face, low quality, unnatural skin",
    "no_mask": "blurry, deformed face, low quality, unnatural skin"
}

In [23]:
for class_name, template in class_prompt_templates.items():
    class_folder = output_dir / class_name
    class_folder.mkdir(parents=True, exist_ok=True)
    for i in tqdm(range(0, images_per_class, batch_size), desc=class_name):
        prompts, negs = [], [negative_prompts[class_name]] * batch_size
        for _ in range(batch_size):
            prompt = template.format(
                shot=random.choice(shot_types),
                gender=random.choice(genders),
                pose=random.choice(poses),
                background=random.choice(backgrounds),
                clothes=random.choice(clothes),
                mask_color=random.choice(mask_colors) if class_name !- "no_mask" else "",
                expression=random.choice(expressions),
            )
            prompts.append(prompt)
        with torch.inference_mode(), torch.autocast("cuda"):
            images = pipe(prompts, negative_pormpt=negs,
                          height=image_size[1],
                          width=image_size[0],
                          num_inference_step=num_inference_steps).images
        for idx, img in enumerate(images):
            img.save(class_folder / f"{class_name}_{i+idx:05}.png")

mask_on:   0%|                                                                                 | 0/125 [00:00<?, ?it/s]C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/50 [00:00<?, ?it/s]

mask_on:   1%|▌                                                                        | 1/125 [00:24<49:59, 24.19s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

mask_on:   2%|█▏                                                                       | 2/125 [00:48<49:59, 24.38s/it]

...

no_mask:  98%|█████████████████████████████████████████████████████████████████████▊ | 123/125 [59:16<00:57, 28.73s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

no_mask:  99%|██████████████████████████████████████████████████████████████████████▍| 124/125 [59:45<00:28, 28.69s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

no_mask: 100%|█████████████████████████████████████████████████████████████████████| 125/125 [1:00:13<00:00, 28.91s/it]


In [24]:
import torch
from xformers.ops import memory_efficient_attention

# 테스트용 텐서 생성 (CUDA로)
q = torch.randn(1, 12, 64, 64, device='cuda', requires_grad=True)
k = torch.randn(1, 12, 64, 64, device='cuda', requires_grad=True)
v = torch.randn(1, 12, 64, 64, device='cuda', requires_grad=True)

try:
    out = memory_efficient_attention(q, k, v)
    print("✅ xFormers CUDA 연산 성공!")
except Exception as e:
    print("❌ xFormers CUDA 연산 실패:", e)

✅ xFormers CUDA 연산 성공!


ImportError: cannot import name 'available_ops' from 'xformers.ops' (C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\xformers\ops\__init__.py)